In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable
from pylab import *
from scipy.stats import wasserstein_distance
import random
import csv
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
# MaryClare's
#os.chdir('/Users/maryclaremartin/Documents/jup/ExtraSensory')

# Josh's
#os.chdir("D:/Research_Projects/REU2021-human-context-recognition/UCI_data")

# Harrison's
os.chdir("/Users/hkimr/Desktop/WPI Github/REU2021-human-context-recognition/UCI_data")

#os.chdir('../')
softmax = nn.Softmax(dim = 1)

In [4]:
##load and scale data
#returns scaled data (X) and labels (Y)
#label: array, list of activities to use (represented by integers)
#users: array, list of users whose data is to be used (represented by integers)

def start_data(label, users, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names

    #X_train = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z'] #Selecting only acceleration columns
    
    #X_train_1 = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z']
    #X_train_2 = data.loc[:,'81 tBodyAccJerk-mean()-X':'160 tBodyGyro-correlation()-Y,Z']
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']

    y_train_subject = pd.read_csv('data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X = GAN_data.iloc[:, :-2].values
    #X = GAN_data.loc[:,'1 tBodyAcc-mean()-X':'160 tBodyGyro-correlation()-Y,Z'].values
    y = GAN_data[['Activity']].values
    
    return X, y

In [5]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim).to(device)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

##calculates generator loss
#gen: generator
#disc: discriminator
#criterion1: loss function1
#criterion2: loss function2
#batch_size: batch size
#z_dim: number of dimensions in the latent space
def get_gen_loss(gen, disc, criterion1, batch_size, z_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    fake_features = gen(latent_vectors)
    
    disc.eval()
    pred_disc = disc(fake_features)
    disc.train()
    
    d_loss = criterion1(pred_disc, torch.ones_like(pred_disc))
        
    return d_loss

In [6]:
def discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

#defines discriminator class
class Discriminator(nn.Module):
    def __init__(self, feature_dim = input_shape, hidden_dim = 16):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            discriminator_block(feature_dim, 25),
            discriminator_block(25, 10),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
    def forward(self, feature_vector):
        return self.disc(feature_vector)
    
def get_disc_loss(gen, disc, criterion, real_features, batch_size, z_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    
    gen.eval()
    fake_features = gen(latent_vectors)
    gen.train()
    
    pred_fake = disc(fake_features.detach())
    
    ground_truth = torch.zeros_like(pred_fake)
    loss_fake = criterion(pred_fake, ground_truth)
    
    pred_real = disc(real_features)
    ground_truth = torch.ones_like(pred_real)
    loss_real = criterion(pred_real, ground_truth)
    
    disc_loss = (loss_fake + loss_real) / 2
    return disc_loss

In [7]:
##calculates performance statistics for each epoch of training
#gen: generator
#disc: discriminator
#b_size: batch size
#z_dim: number of dimensions of the latent space
##returns accuracy, precision, recall, fpR, and f1 score
def performance_stats(gen, disc, b_size, z_dim, batch = None):
    tp = 0 #true positive
    fp = 0 #false positive
    tn = 0 #true negative
    fn = 0 #false negative

    with torch.no_grad():
        y_hat_fake = torch.Tensor([0])
        y_hat_real = torch.Tensor([0])
        
        if batch is None:
            print('Error')
        else:
            latent_vectors = get_noise(b_size, z_dim)
            gen.eval()
            fake_features = gen(latent_vectors)
            gen.train()
            disc.eval()
            y_hat_fake = torch.round(disc(fake_features))
            y_hat_real = torch.round(disc(batch))
            disc.train()
         
        fpR = torch.mean(y_hat_fake)
        recall = torch.mean(y_hat_real)
        class_acc = ((len(y_hat_fake) - torch.sum(y_hat_fake)) + torch.sum(y_hat_real)) / (2*b_size)
        #print(f'Classification Accuracy: {class_acc:.2f}'
        #print(f'Recall: {recall:.2f}') #What percent of the true positives were identified
        return class_acc, recall, fpR
    
def save_model(gen, disc, model_name):
    torch.save(gen.state_dict(), f"saved_models/{model_name}_gen.param")
    torch.save(disc.state_dict(), f"saved_models/{model_name}_disc.param")

In [8]:
###initalize parameters that depend on training loop parameters
#X: acceleration data
#y: labels associated with X data (fake or real)
#z_dim: number of dimensions to the latent space
#disc_lr: discriminator learning rate
#gen_lr: generator learning rate
#DISCRIMINATOR: 1 to indicate if discriminator is training
#batch_size: batch size
#disc: initialized discrimiantor

def initialize_params(X, y, z_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc):
    #initialize generator
    gen = Generator(z_dim).to(device)
    #indicate that discriminator is training
    to_train = DISCRIMINATOR
    #create training features
    train_features = torch.tensor(X)
    #create training labels
    train_labels = torch.tensor(y)
    #concatenate to create training data
    train_data = torch.utils.data.TensorDataset(train_features, train_labels)
    #create data loader for training data
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
    #initialize generator and discriminator optimizers
    opt_disc = optim.Adam(disc.parameters(), lr = disc_lr)
    opt_gen = optim.Adam(gen.parameters(), lr = gen_lr)
    
    return gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen   

In [9]:
######Training loop to train GAN

#Parameters to specifiy: 
    #X: starting accelerometer data
    #y: starting labels for X data (fake or real)
    
#Set parameters (do not change)
    #criterion: loss function (BCE)
    #dig: number of significant digits for printing (5)
    #feature_dim: Number of dimensions of output from generator (26)
    #GENERATOR: set generator to zero for training
    #DISCRIMINATOR: set discriminator to one for training
    #train_string: starting machine to train (DISC)
    #disc: initalize discriminator
    #rel_epochs: Epochs passed since last switch (constant training) (0)
    #rows: initialization of array to save data of each epoch to CSV file ([])
    #heading: array of column headings for table (["Epoch", "Machine Training", "Discriminator Loss", 
                    #"Generator Loss", "FPR", "Recall", "Median Wasserstein", "Mean Wasserstein"])
    #table: intialize a table as a pretty table to save epoch data
    #switch_count: number of switches in dynamic training (0)
    
#Set parameters (can change):
    #z_dim: number of dimensions of latent vector (100)
    #gen_lr: generator learning rate (.001)
    #disc_lr: discriminator learning rate (.001) (shoud be equal to gen_lr)
    #batch_size: batch size (75)
    #print_batches: Show model performance per batch (False)
    #n_epochs: number of epochs to train (100)
    #constant_train_flag: (False)
        #Set to true to train based on constant # of epochs per machine 
        #Set to false to train dynamically based on machine performance
        
    #Constant training approach:
        #disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (5)
        #gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #epoch_threshold: Epoch number to change training epoch ratio (50)
        #disc_epochs_change: New number of consecutive epochs to train discriminator after epoch threshold is exceeded (1)
        #gen_epochs_change: New number of consecutive epochs to train generator after epoch threshold is exceeded (50)
    
    #Dynamic training approach:                        
        #static_threshold: Epoch number to change from static ratio to dynamic (18)
        #static_disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (4)
        #static_gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #pull_threshold: Accuracy threshold for switching machine training when the generator is no longer competitive (0.4)
        #push_threshold: Accuracy threshold for switching machine training when the discriminator is no longer competitive (0.6)
        #recall_threshold: threshold for recall to switch machine training when discriminator is training well
        #switch_flag: indicates if we should switch our training machine (False)
        
def training_loop(X, y, criterion1 = nn.BCELoss(), gan_id = "Mod Test Gan", dig = 5, feature_dim = input_shape, 
                  GENERATOR = 0, DISCRIMINATOR = 1, train_string = "DISC", disc = Discriminator(), z_dim = 100,
                  gen_lr =  0.001, disc_lr = 0.001, batch_size = 100, constant_train_flag = False, disc_epochs = 5,
                  gen_epochs = 2, epoch_threshold = 50, disc_epochs_change = 5, gen_epochs_change = 2, rel_epochs = 0,
                 static_threshold = 28, static_disc_epochs = 5, static_gen_epochs = 2, pull_threshold = 0.3,
                 push_threshold = 0.7, recall_threshold = 0.75, n_epochs = 1000, rows = [],
                switch_flag = False, switch_count = 0, last_real_features = []):
    
    disc.to(device)
    #returns generator, sets discriminator training, creates training tensor, loads data, and initializes optimizers
    gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen = initialize_params(X, y, z_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc)
    
    #visualize_gen(X, gen, z_dim, a_dim, u_dim)
    gen_epochs = 0
    
    last_D_loss = -1.0
    last_G_loss = -1.0
    
    mean_mean = []
    mean_median = []
    
    for epoch in range(n_epochs):  
        if constant_train_flag:
            if to_train == DISCRIMINATOR and rel_epochs >= disc_epochs:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

            elif to_train == GENERATOR and rel_epochs >= gen_epochs:
                rel_epochs = 0
                to_train = DISCRIMINATOR
                train_string = "DISC"

            # Change epoch ratio after intial 'leveling out'
            if epoch == epoch_threshold:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

                old_ratio = gen_epochs / disc_epochs
                gen_epochs = gen_epochs_change
                disc_epochs = disc_epochs_change
                new_ratio = gen_epochs / disc_epochs
                print(f'\n\nTraining ratio of G/D switched from {old_ratio:.{dig}f} to {new_ratio:.{dig}f}\n\n')
        else:
            if epoch < static_threshold:
                if to_train == DISCRIMINATOR and rel_epochs >= static_disc_epochs:
                    rel_epochs = 0
                    to_train = GENERATOR
                    train_string = "GEN"

                elif to_train == GENERATOR and rel_epochs >= static_gen_epochs:
                    rel_epochs = 0
                    to_train = DISCRIMINATOR
                    train_string = "DISC"

            else:
                if not switch_flag:
                    print("\nSwitching to Dynamic Training\n")
                    switch_flag = True
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                if to_train == DISCRIMINATOR and fpR <= pull_threshold and R >= recall_threshold:
                    to_train = GENERATOR
                    train_string = "GEN"
                    print("\nPull Generator\n")
                    switch_count += 1
                if to_train == GENERATOR and fpR >= push_threshold:
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                    print("\nPush Generator\n")
                    switch_count += 1
                    
        print(f'Epoch[{epoch + 1}/{n_epochs}] Train: {train_string} ', end ='')
        
        for batch_idx, (real_features, _) in enumerate(train_loader):
            #Size of this current batch
            batch_size = len(real_features)
            #Send batch to GPU
            real_features = real_features.float().to(device)
            
            if to_train == DISCRIMINATOR:
                ### Training Discriminator
                #visualize_real_batch(real_features.float())
                opt_disc.zero_grad()
                disc_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim)
                disc_loss.backward(retain_graph = True)
                opt_disc.step()
                last_D_loss = disc_loss.item()
                # If generator has yet to ever train, run intial evaluation
                if last_G_loss == -1.0:
                    last_G_loss = get_gen_loss(gen, disc, criterion1, batch_size, z_dim) 
            else:
                ### Training Generator
                opt_gen.zero_grad()
                gen_loss = get_gen_loss(gen, disc, criterion1, batch_size, z_dim)
                gen_loss.backward()
                opt_gen.step()
                last_G_loss = gen_loss.item()
                # If discriminator has yet to ever train, run intial evaluation
                if last_D_loss == -1.0:
                    last_D_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim)
                
            if batch_idx == (len(train_loader) - 1):
                acc, R, fpR = performance_stats(gen, disc, batch_size, z_dim, batch = real_features)
                
        mean_mean_w = torch.mean(torch.Tensor(mean_mean)) 
        mean_median_w = torch.mean(torch.Tensor(mean_median))
            
        ### Currently doesn't print Median/Mean Wasserstein --> Change if needed
        print(f'| LossD: {last_D_loss:.{dig}f}, LossG: {last_G_loss:.{dig}f} | Acc: {acc:.{dig}f} | fpR: {fpR:.{dig}f} | R: {R:.{dig}f}')
    
        if to_train == GENERATOR:
            gen_epochs += 1
            
        mean_mean.clear()
        mean_median.clear()
        rel_epochs += 1
        
        
    print("\n\nTraining Session Finished")
    print(f"Encountered {switch_count} non-trivial training swaps")
    percent = gen_epochs / n_epochs
    print(f"Trained Generator {gen_epochs} out of {n_epochs} ({percent:.3f})")
  
    print("Model Results Sucessfully Saved to \"model_outputs/" + gan_id + ".txt\"")

    save_model(gen, disc, gan_id)
    print("Generator's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_gen.param\"")
    print("Discriminators's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_disc.param\"")
    save_model(gen, disc, gan_id)
    
    

In [10]:
X, y = start_data([1], [5], sub_features, act_features) 

#Train
training_loop(X, y, gan_id="U2A0 Solo GAN", batch_size = len(X), gen_lr=.0001, disc_lr =.0001, n_epochs=30000, dig=5, constant_train_flag=False)

Epoch[1/30000] Train: DISC | LossD: 0.69972, LossG: 0.73084 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[2/30000] Train: DISC | LossD: 0.69929, LossG: 0.73084 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[3/30000] Train: DISC | LossD: 0.69823, LossG: 0.73084 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[4/30000] Train: DISC | LossD: 0.69768, LossG: 0.73084 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[5/30000] Train: DISC | LossD: 0.69757, LossG: 0.73084 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[6/30000] Train: GEN | LossD: 0.69757, LossG: 0.72922 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[7/30000] Train: GEN | LossD: 0.69757, LossG: 0.72897 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[8/30000] Train: DISC | LossD: 0.69715, LossG: 0.72897 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[9/30000] Train: DISC | LossD: 0.69699, LossG: 0.72897 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[10/30000] Train: DISC | LossD: 0.69828, LossG: 0.72897 | Acc: 0.50000

Epoch[90/30000] Train: DISC | LossD: 0.67706, LossG: 0.72712 | Acc: 0.75893 | fpR: 0.00000 | R: 0.51786
Epoch[91/30000] Train: DISC | LossD: 0.67523, LossG: 0.72712 | Acc: 0.85714 | fpR: 0.00000 | R: 0.71429
Epoch[92/30000] Train: DISC | LossD: 0.67323, LossG: 0.72712 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929

Pull Generator

Epoch[93/30000] Train: GEN | LossD: 0.67323, LossG: 0.72860 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929
Epoch[94/30000] Train: GEN | LossD: 0.67323, LossG: 0.72856 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929
Epoch[95/30000] Train: GEN | LossD: 0.67323, LossG: 0.72775 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929
Epoch[96/30000] Train: GEN | LossD: 0.67323, LossG: 0.72790 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929
Epoch[97/30000] Train: GEN | LossD: 0.67323, LossG: 0.72749 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929
Epoch[98/30000] Train: GEN | LossD: 0.67323, LossG: 0.72719 | Acc: 0.91964 | fpR: 0.00000 | R: 0.83929
Epoch[99/30000] Train: GEN | LossD: 0.67323, LossG: 0

Epoch[190/30000] Train: GEN | LossD: 0.67323, LossG: 0.69656 | Acc: 0.82143 | fpR: 0.19643 | R: 0.83929
Epoch[191/30000] Train: GEN | LossD: 0.67323, LossG: 0.69804 | Acc: 0.84821 | fpR: 0.14286 | R: 0.83929
Epoch[192/30000] Train: GEN | LossD: 0.67323, LossG: 0.69659 | Acc: 0.79464 | fpR: 0.25000 | R: 0.83929
Epoch[193/30000] Train: GEN | LossD: 0.67323, LossG: 0.69602 | Acc: 0.83036 | fpR: 0.17857 | R: 0.83929
Epoch[194/30000] Train: GEN | LossD: 0.67323, LossG: 0.69566 | Acc: 0.77679 | fpR: 0.28571 | R: 0.83929
Epoch[195/30000] Train: GEN | LossD: 0.67323, LossG: 0.69529 | Acc: 0.77679 | fpR: 0.28571 | R: 0.83929
Epoch[196/30000] Train: GEN | LossD: 0.67323, LossG: 0.69534 | Acc: 0.81250 | fpR: 0.21429 | R: 0.83929
Epoch[197/30000] Train: GEN | LossD: 0.67323, LossG: 0.69502 | Acc: 0.72321 | fpR: 0.39286 | R: 0.83929
Epoch[198/30000] Train: GEN | LossD: 0.67323, LossG: 0.69431 | Acc: 0.75893 | fpR: 0.32143 | R: 0.83929
Epoch[199/30000] Train: GEN | LossD: 0.67323, LossG: 0.69401 | A

Epoch[271/30000] Train: GEN | LossD: 0.67999, LossG: 0.69427 | Acc: 0.84821 | fpR: 0.30357 | R: 1.00000
Epoch[272/30000] Train: GEN | LossD: 0.67999, LossG: 0.69321 | Acc: 0.85714 | fpR: 0.28571 | R: 1.00000
Epoch[273/30000] Train: GEN | LossD: 0.67999, LossG: 0.69317 | Acc: 0.83036 | fpR: 0.33929 | R: 1.00000
Epoch[274/30000] Train: GEN | LossD: 0.67999, LossG: 0.69209 | Acc: 0.86607 | fpR: 0.26786 | R: 1.00000
Epoch[275/30000] Train: GEN | LossD: 0.67999, LossG: 0.69259 | Acc: 0.83036 | fpR: 0.33929 | R: 1.00000
Epoch[276/30000] Train: GEN | LossD: 0.67999, LossG: 0.69230 | Acc: 0.83929 | fpR: 0.32143 | R: 1.00000
Epoch[277/30000] Train: GEN | LossD: 0.67999, LossG: 0.69137 | Acc: 0.79464 | fpR: 0.41071 | R: 1.00000
Epoch[278/30000] Train: GEN | LossD: 0.67999, LossG: 0.69161 | Acc: 0.80357 | fpR: 0.39286 | R: 1.00000
Epoch[279/30000] Train: GEN | LossD: 0.67999, LossG: 0.69214 | Acc: 0.75000 | fpR: 0.50000 | R: 1.00000
Epoch[280/30000] Train: GEN | LossD: 0.67999, LossG: 0.69167 | A

Epoch[374/30000] Train: GEN | LossD: 0.66782, LossG: 0.68232 | Acc: 0.63393 | fpR: 0.73214 | R: 1.00000

Push Generator

Epoch[375/30000] Train: DISC | LossD: 0.67242, LossG: 0.68232 | Acc: 0.67857 | fpR: 0.64286 | R: 1.00000
Epoch[376/30000] Train: DISC | LossD: 0.67304, LossG: 0.68232 | Acc: 0.66964 | fpR: 0.66071 | R: 1.00000
Epoch[377/30000] Train: DISC | LossD: 0.67090, LossG: 0.68232 | Acc: 0.68750 | fpR: 0.62500 | R: 1.00000
Epoch[378/30000] Train: DISC | LossD: 0.67494, LossG: 0.68232 | Acc: 0.68750 | fpR: 0.62500 | R: 1.00000
Epoch[379/30000] Train: DISC | LossD: 0.66807, LossG: 0.68232 | Acc: 0.71429 | fpR: 0.57143 | R: 1.00000
Epoch[380/30000] Train: DISC | LossD: 0.66960, LossG: 0.68232 | Acc: 0.69643 | fpR: 0.60714 | R: 1.00000
Epoch[381/30000] Train: DISC | LossD: 0.66785, LossG: 0.68232 | Acc: 0.71429 | fpR: 0.57143 | R: 1.00000
Epoch[382/30000] Train: DISC | LossD: 0.66982, LossG: 0.68232 | Acc: 0.67857 | fpR: 0.64286 | R: 1.00000
Epoch[383/30000] Train: DISC | LossD: 0

...


Push Generator

Epoch[29555/30000] Train: DISC | LossD: 0.65544, LossG: 0.63238 | Acc: 0.58929 | fpR: 0.78571 | R: 0.96429
Epoch[29556/30000] Train: DISC | LossD: 0.67968, LossG: 0.63238 | Acc: 0.58929 | fpR: 0.78571 | R: 0.96429
Epoch[29557/30000] Train: DISC | LossD: 0.65154, LossG: 0.63238 | Acc: 0.62500 | fpR: 0.71429 | R: 0.96429
Epoch[29558/30000] Train: DISC | LossD: 0.64752, LossG: 0.63238 | Acc: 0.60714 | fpR: 0.73214 | R: 0.94643
Epoch[29559/30000] Train: DISC | LossD: 0.68040, LossG: 0.63238 | Acc: 0.60714 | fpR: 0.69643 | R: 0.91071
Epoch[29560/30000] Train: DISC | LossD: 0.65044, LossG: 0.63238 | Acc: 0.60714 | fpR: 0.66071 | R: 0.87500
Epoch[29561/30000] Train: DISC | LossD: 0.67535, LossG: 0.63238 | Acc: 0.50000 | fpR: 0.83929 | R: 0.83929
Epoch[29562/30000] Train: DISC | LossD: 0.67322, LossG: 0.63238 | Acc: 0.54464 | fpR: 0.69643 | R: 0.78571
Epoch[29563/30000] Train: DISC | LossD: 0.67552, LossG: 0.63238 | Acc: 0.54464 | fpR: 0.66071 | R: 0.75000
Epoch[29564/30000] T

Epoch[29651/30000] Train: GEN | LossD: 0.63089, LossG: 0.77102 | Acc: 0.66964 | fpR: 0.48214 | R: 0.82143
Epoch[29652/30000] Train: GEN | LossD: 0.63089, LossG: 0.74989 | Acc: 0.60714 | fpR: 0.60714 | R: 0.82143
Epoch[29653/30000] Train: GEN | LossD: 0.63089, LossG: 0.74081 | Acc: 0.58036 | fpR: 0.66071 | R: 0.82143
Epoch[29654/30000] Train: GEN | LossD: 0.63089, LossG: 0.75181 | Acc: 0.57143 | fpR: 0.67857 | R: 0.82143
Epoch[29655/30000] Train: GEN | LossD: 0.63089, LossG: 0.74662 | Acc: 0.60714 | fpR: 0.60714 | R: 0.82143
Epoch[29656/30000] Train: GEN | LossD: 0.63089, LossG: 0.69924 | Acc: 0.63393 | fpR: 0.55357 | R: 0.82143
Epoch[29657/30000] Train: GEN | LossD: 0.63089, LossG: 0.71219 | Acc: 0.59821 | fpR: 0.62500 | R: 0.82143
Epoch[29658/30000] Train: GEN | LossD: 0.63089, LossG: 0.68457 | Acc: 0.54464 | fpR: 0.73214 | R: 0.82143

Push Generator

Epoch[29659/30000] Train: DISC | LossD: 0.70708, LossG: 0.68457 | Acc: 0.54464 | fpR: 0.73214 | R: 0.82143
Epoch[29660/30000] Train: DI

Epoch[29743/30000] Train: DISC | LossD: 0.62834, LossG: 0.75100 | Acc: 0.59821 | fpR: 0.75000 | R: 0.94643
Epoch[29744/30000] Train: DISC | LossD: 0.62787, LossG: 0.75100 | Acc: 0.66071 | fpR: 0.62500 | R: 0.94643
Epoch[29745/30000] Train: DISC | LossD: 0.60801, LossG: 0.75100 | Acc: 0.67857 | fpR: 0.60714 | R: 0.96429
Epoch[29746/30000] Train: DISC | LossD: 0.61087, LossG: 0.75100 | Acc: 0.62500 | fpR: 0.71429 | R: 0.96429
Epoch[29747/30000] Train: DISC | LossD: 0.65532, LossG: 0.75100 | Acc: 0.72321 | fpR: 0.51786 | R: 0.96429
Epoch[29748/30000] Train: DISC | LossD: 0.63613, LossG: 0.75100 | Acc: 0.69643 | fpR: 0.55357 | R: 0.94643
Epoch[29749/30000] Train: DISC | LossD: 0.63205, LossG: 0.75100 | Acc: 0.70536 | fpR: 0.51786 | R: 0.92857
Epoch[29750/30000] Train: DISC | LossD: 0.61681, LossG: 0.75100 | Acc: 0.68750 | fpR: 0.53571 | R: 0.91071
Epoch[29751/30000] Train: DISC | LossD: 0.63802, LossG: 0.75100 | Acc: 0.69643 | fpR: 0.50000 | R: 0.89286
Epoch[29752/30000] Train: DISC | Loss


Pull Generator

Epoch[29843/30000] Train: GEN | LossD: 0.63671, LossG: 0.75451 | Acc: 0.64286 | fpR: 0.55357 | R: 0.83929
Epoch[29844/30000] Train: GEN | LossD: 0.63671, LossG: 0.75467 | Acc: 0.65179 | fpR: 0.53571 | R: 0.83929
Epoch[29845/30000] Train: GEN | LossD: 0.63671, LossG: 0.74397 | Acc: 0.61607 | fpR: 0.60714 | R: 0.83929
Epoch[29846/30000] Train: GEN | LossD: 0.63671, LossG: 0.68210 | Acc: 0.55357 | fpR: 0.73214 | R: 0.83929

Push Generator

Epoch[29847/30000] Train: DISC | LossD: 0.69809, LossG: 0.68210 | Acc: 0.54464 | fpR: 0.75000 | R: 0.83929
Epoch[29848/30000] Train: DISC | LossD: 0.67876, LossG: 0.68210 | Acc: 0.57143 | fpR: 0.69643 | R: 0.83929
Epoch[29849/30000] Train: DISC | LossD: 0.69148, LossG: 0.68210 | Acc: 0.57143 | fpR: 0.69643 | R: 0.83929
Epoch[29850/30000] Train: DISC | LossD: 0.64454, LossG: 0.68210 | Acc: 0.62500 | fpR: 0.60714 | R: 0.85714
Epoch[29851/30000] Train: DISC | LossD: 0.67231, LossG: 0.68210 | Acc: 0.58036 | fpR: 0.67857 | R: 0.83929
Epoch[2

Epoch[29939/30000] Train: GEN | LossD: 0.62082, LossG: 0.68555 | Acc: 0.48214 | fpR: 0.78571 | R: 0.75000

Push Generator

Epoch[29940/30000] Train: DISC | LossD: 0.69406, LossG: 0.68555 | Acc: 0.46429 | fpR: 0.82143 | R: 0.75000
Epoch[29941/30000] Train: DISC | LossD: 0.72010, LossG: 0.68555 | Acc: 0.52679 | fpR: 0.69643 | R: 0.75000
Epoch[29942/30000] Train: DISC | LossD: 0.69639, LossG: 0.68555 | Acc: 0.51786 | fpR: 0.71429 | R: 0.75000
Epoch[29943/30000] Train: DISC | LossD: 0.70544, LossG: 0.68555 | Acc: 0.56250 | fpR: 0.64286 | R: 0.76786
Epoch[29944/30000] Train: DISC | LossD: 0.69971, LossG: 0.68555 | Acc: 0.57143 | fpR: 0.66071 | R: 0.80357
Epoch[29945/30000] Train: DISC | LossD: 0.67909, LossG: 0.68555 | Acc: 0.50893 | fpR: 0.78571 | R: 0.80357
Epoch[29946/30000] Train: DISC | LossD: 0.67740, LossG: 0.68555 | Acc: 0.57143 | fpR: 0.67857 | R: 0.82143
Epoch[29947/30000] Train: DISC | LossD: 0.67916, LossG: 0.68555 | Acc: 0.63393 | fpR: 0.55357 | R: 0.82143
Epoch[29948/30000] Tr